Formation OpenClassrooms DS-IML

**Projet 2**

Par **Marc Lefèvre**, <marc.lefevre@noos.fr>

## 6ème Partie

# Moteur de Recommandation

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

Récupération des données et outils divers

In [2]:
df = pd.read_csv("Data/df.csv")
vect = pickle.load(open("Data/vect.pickle", "rb"))
matrix = pickle.load(open("Data/matrix.pickle", "rb"))
tokenizer = pickle.load(open("Data/tokenizer.pickle", "rb"))

sw = stopwords.words("french")

stemmer = FrenchStemmer()

def sw_stem_str(l):
    
    for elt in l:
        if elt in sw :
            l.remove(elt)

    li = [stemmer.stem(w) for w in l]
        
    ligne = " ".join(li)
    return ligne

**Fonctionnement du moteur** :<br>Dans un premier temps, la requête utilisateur, une fois saisie, sera traitée de la même manière que les données de notre colonne **tokens** (mise en caractères minuscules, tokenisation, stop words, stemming...).

### Saisie et traitement du texte d'une requête

In [3]:
input_text = input("Entrez votre recherche :")
entry = sw_stem_str(tokenizer.tokenize(input_text.lower()))
print(f"Texte de la requête traité : {entry}")

entrez votre recherche : Coke Light


Texte de la requête traité : cok light


La requête est ensuite vectorisée par le même vectorisateur que précédemment :

In [4]:
entry_v = vect.transform([entry])

On crée maintenant une nouvelle colonne **score** au dataset. Cette colonne contient le score de **similarité cosinusoidale** entre la requête et chaque ligne de notre matrice (dont chaque ligne est la donnée texte **tokens** vectorisée).<br>Ainsi, pour chaque produit du dataset, on se retrouve avec son **score de pertinence sémantique** avec la requête utilisateur.<br><br>**Rappel** : La méthode **cosine_similarity** de **sklearn.metrics.pairwise** calcule la similarité cosinusoidale entre deux vecteurs. Dans notre cas, c'est une manière efficace de mesurer la proximité sémantique entre deux textes.<br>Ce score se situe entre **0** et **1**, plus il est proche de **1** plus la similarité est grande.

In [5]:
df["score"] = cosine_similarity(entry_v, matrix)[0]

In [6]:
df[["product_name", "score"]].sort_values("score", ascending = False).head(10).style.hide_index()

product_name,score
coke,0.82181
diet coke,0.770238
coke zero,0.669702
diet coke,0.659333
coke diet,0.659333
diet coke,0.659333
diet coke,0.659333
diet coke,0.659333
diet coke,0.659333
cokies pur beurre mini,0.623943


Dans notre exemple, on renvoie les dix réponses les plus pertinentes à la requête **"Coke Light"** en affichant ici l'index, le nom, et le score du produit. Le choix des infos par produit à retourner, parmi tout ce qui est disponible dans le dataset, est à déterminer ultérieurement.

### Filtres

On peut également définir un système de **filtres** permettant à l'utilisateur de restreindre le champ de ses recherches en fonctions de certains critères en fonction des données dont on dispose.<br>Par exemple, rechercher (sous réserve qu'on ait assez de données) :<br>- au sein d'une certaine classe du **nutriscore_grade** ou du **nova_group**.<br>- des produits sans **allergènes**, **alcohol**, **huile de palme**...<br>- des produits contenant tel ou tel **nutriment**...<br>**Etc.**

En pratique, cela reviendra simplement à afficher les résultats de requêtes depuis une version filtrée du dataset.<br>Le choix éventuel du filtre se ferait via un élément d'interface aboutissant, par exemple, à la saisie d'une clé d'un "dictionnaire de filtres".

## Prototype de moteur

Vous trouverez ci-dessous une démonstration de notre *embryon* de moteur. Pour le faire marcher, sous réserve d'avoir tout ce qu'il faut (données originales modifiées par le script **P2_Pipeline.py**), il suffit d'invoquer la classe **moteur** et de jouer avec ses méthodes **query** (pour entrer une requete et recevoir ses résultats) ou **set_filter** (pour tester les filtres de *démonstration*).<br> On peut aussi modifier les paramètres présents sous forme de **constantes** au début du code.

**Syntaxe** :<br>- Création du moteur : mot = Moteur()<br>- Requête : mot.query()<br>- Choix d'un filtre : mot.set_filter()



In [18]:
# Nombre de résultats à afficher
NB_RESULTS = 15

# Liste des colonnes de données dans les réponses du moteur
LISTE_COL = ["product_name", "nutriscore_grade", "nova_group", "fiber_100g", "score"]

# Classe définissant notre moteur de recommandation

class Moteur():
    
    # Initialisation de la classe
    def __init__(self):
        
        self.df = pd.read_csv("Data/df.csv")
        self.df_req = self.df.copy()
        self.vect = pickle.load(open("Data/vect.pickle", "rb"))
        self.matrix = pickle.load(open("Data/matrix.pickle", "rb"))
        self.tokenizer = pickle.load(open("Data/tokenizer.pickle", "rb"))
        self.entry = None
        self.filter_idx = None
        self.apply_filter = False
        self.sw = stopwords.words("french")
        self.stemmer = FrenchStemmer()
        self.input_text = None
        self.need_reinit = False
        
        # Quelques exemples de filtres pré-définis en démonstration
        self.filters = {"1":(self.df["nutriscore_grade"] == "a"), "2":(self.df["nova_group"] == 1.0), 
                   "3":(self.df["fiber_100g"].notnull()) & (self.df["fiber_100g"] > 0)}
        
    # méthode de traitement du texte de requête
    def sw_stem_str(self, l):
    
        for elt in l:
            if elt in self.sw :
                l.remove(elt)

        li = [self.stemmer.stem(w) for w in l]

        ligne = " ".join(li)
        return ligne
        
    # Méthode permettant d'activer un filtre
    def set_filter(self):
        
        texte = ""
    
        while texte not in ["1", "2", "3", "4"]:
            print("1 - Nutriscore = A")
            print("2 - Aliment non trasformé : Nova Group = 1")
            print("3 - Contient des fibres")
            print("4 - Non, aucun filtre")
            texte = str(input("Choisissez votre filtre :"))
    
        if texte in ["1", "2", "3"] :
        
            self.filter_idx = texte
            self.apply_filter = True
    
        else :
            self.filter_idx = None
            self.apply_filter = False
            
    # méthode permettant d'entrée une requête et de recevoir le résultat du moteur
    def query(self):
        
        if self.need_reinit :
            self.df_req = self.df.copy()
            self.need_reinit = False
        
        self.input_text = input("Entrez votre recherche :")
        
        self.entry = self.vect.transform([self.sw_stem_str(self.tokenizer.tokenize(self.input_text.lower()))])        
        
        self.df_req["score"] = cosine_similarity(self.entry, self.matrix)[0]

        if self.apply_filter :
            self.df_req = self.df_req[self.filters[self.filter_idx]]
            self.apply_filter = False
            self.need_reinit = True

        return self.df_req[LISTE_COL].sort_values("score", 
                                                  ascending = False).head(NB_RESULTS).style.hide_index()

In [19]:
mot = Moteur()

In [21]:
mot.set_filter()

1 - Nutriscore = A
2 - Aliment non trasformé : Nova Group = 1
3 - Contient des fibres
4 - Non, aucun filtre


Choisissez votre filtre : 2


In [22]:
mot.query()

Entrez votre recherche : raisin


product_name,nutriscore_grade,nova_group,fiber_100g,score
raisin,e,1,nan,1
jus de raisin,e,1,0,0.769837
jus de raisin,e,1,0,0.769837
le jus - raisin,e,1,nan,0.769837
raisin secs,nan,1,nan,0.730936
jus de raisin,e,1,nan,0.669604
jus de raisin,e,1,nan,0.669604
jus de raisin,e,1,0.1,0.661258
100% jus raisin,e,1,0,0.654863
raisin de france,e,1,nan,0.63299


# Conclusion et améliorations possibles